In [1]:
import numpy as np
import os.path
import logging
import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot as plt
from matplotlib import rc
import pickle
from datetime import date

# from util import *
# from hilbert import hilbert_xxz
# from spectral_net import *
from config import deepqd_config
from DQD_leftover import *
# from VanillaQD import VanillaQD
from ClsQuantumDynamics import QuantumDynamics as clsQD


T = 32*10
directory =  '/home/bingjiay/gpfs/ControlQD'
# config_dict = directory + '/config/20-10-16-19-30-28s16n16-20b16-20MC4000hd128-cell4-lr0.001_skip*4NN0'
# config_dict = directory + '/config/20-10-16-15-15-45s16n32-20b32-20MC4000hd128-cell4-lr0.001_skip*4NN0'
# config_dict = directory + '/config/20-10-16-15-16-00s16n64-20b64-20MC4000hd128-cell4-lr0.001_skip*4NN0'

# config_dict = directory + '/config/20-10-12-02-47-58s24n48-20b48-20MC4000hd128-cell4-lr0.001_skip*4NN0'
# config_dict = directory + '/config/20-10-12-02-26-46s24n24-20b24-20MC4000hd128-cell4-lr0.001_skip*4NN0'

# config_dict = directory + '/config/20-10-12-02-48-24s32n32-20b32-20MC4000hd128-cell4-lr0.001_skip*4NN0'
# config_dict = directory + '/config/20-10-13-02-43-27s32n48-20b48-20MC4000hd128-cell4-lr0.001_skip*4NN0'
## wall
# config_dict = directory + '/config/20-10-19-00-35-59s16n32-20b32-20MC4000hd128-cell4-lr0.001_wallNN0'

# config_dict = directory + '/config/20-10-19-01-48-31s24n24-20b24-20MC4000hd128-cell4-lr0.001_wallNN0'
# config_dict = directory + '/config/20-10-19-10-18-10s24n24-20b48-20MC8000hd128-cell4-lr0.001_wallNN0'

config_dict = directory + '/config/20-10-19-01-49-39s32n32-20b32-20MC4000hd128-cell4-lr0.001_wallNN0'
# config_dict = directory + '/config/20-10-19-11-29-04s32n32-20b64-20MC4000hd128-cell4-lr0.001_wallNN0'
# config_dict = directory + '/config/20-10-21-20-19-14s32n32-20b32-20MC8000hd128-cell4-lr0.001_wall-lowlyingNN0'
# config_dict = directory + '/config/20-10-22-04-10-26s32n32-20b96-20MC4000hd128-cell4-lr0.001_wallNN0'



# config_dict = directory + '/config/20-10-19-21-29-00s40n20-20b40-20MC8000hd128-cell4-lr0.001_wallNN0'
# config_dict = directory + '/config/20-10-20-10-36-32s40n20-20b40-20MC4000hd128-cell4-lr0.001_wall-losscutNN0'
# config_dict = directory + '/config/20-10-20-12-58-55s40n20-20b80-20MC4000hd128-cell4-lr0.001_wall-losscutNN0'
# config_dict = directory + '/config/20-10-20-14-56-52s40n20-20b20-20MC4000hd128-cell4-lr0.001_wall-losscutNN0'
# config_dict = directory + '/config/20-10-20-16-35-18s40n10-20b40-20MC8000hd128-cell4-lr0.001_wall-losscutNN0'

# config_dict = directory + '/config/20-10-21-12-54-54s40n20-20b20-20MC10000hd128-cell4-lr0.001_wall-lowlyingNN0'


### Initialization
args = deepqd_config()
with open(config_dict,'rb') as f:
    args.__dict__ = pickle.load(f)

if th.cuda.is_available():
    args.device = 'cuda'
    args.num_device = 2
else:
    args.device = 'cpu'
#####
# data_dir = './nn_data/' + args.experiment_name
output_dir = './output/' + args.experiment_name
# if not os.path.exists(data_dir):
#     os.mkdir(data_dir)
#     print("Directory " , data_dir ,  " Created ")
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
    print("Directory " , output_dir ,  " Created ")



In [2]:
nbatch = 12000
reps = 20
stype='wall'
for depth in [0]:
    deepqd = spectral_tree(args)
    ham = deepqd.hilbert
    exact_sampling = True if ham.Lx<20 else False
    out_appdix = 'exact.png' if exact_sampling else 'mc.png'

    try:
        with open('./benchmark/{}Jz-1L32T10.0.tenpy'.format(stype),'rb') as file:
            ref_Jt = pickle.load(file)
            ref_entropy = pickle.load(file)
            ref_szsz_avg = pickle.load(file)
            ref_stagger_avg = pickle.load(file)
            ref_sdw_avg = pickle.load(file)
    except:
        print('can not load benchmarking data')

    if depth>0:
        deepqd.load_tree(directory)
        deepqd.load_all_model(directory=directory)
    else:
        deepqd.load_model('0',directory=directory)
    time = np.arange(101) / 100 * T
    entropy_list = []
    nn_coupling_list = []
    spin_imbalance_list = []
    mirror_szsz_list = []
    spin_density = []
    for t in time:
        print('working on t={}, exact={}'.format(t,exact_sampling))
        if exact_sampling:
            entropy, szsz_avg, stagger_avg, sdw_avg = deepqd.get_observation(depth,t,exact=True)
        else:
            entropy ,szsz_avg, mirror_szsz_avg, stagger_avg, sdw_avg = 0, 0, 0,0, 0
            for q in range(reps):
                _, szsz, mirror_szsz, stagger, sdw = deepqd.get_observation(depth=depth,t=t,batchsize=nbatch)
                szsz_avg += szsz / reps
                mirror_szsz_avg += mirror_szsz / reps
                stagger_avg += stagger / reps
                sdw_avg += sdw / reps
        entropy_list.append(entropy)
        nn_coupling_list.append(szsz_avg)
        mirror_szsz_list.append(mirror_szsz_avg)
        spin_imbalance_list.append(stagger_avg)
        spin_density.append(sdw_avg)

    entropy = np.array(entropy_list)
    nn_coupling = np.array(nn_coupling_list)
    mirror_szsz = np.array(mirror_szsz_list)
    spin_imbalance = np.array(spin_imbalance_list)
    spin_density = np.array(spin_density)


Loaded model from /home/bingjiay/gpfs/ControlQD/nn_data/s32n32-20b32-20MC4000hd128-cell4-lr0.001_wall/0.net
working on t=0.0, exact=False
working on t=3.2, exact=False
working on t=6.4, exact=False
working on t=9.6, exact=False
working on t=12.8, exact=False
working on t=16.0, exact=False
working on t=19.2, exact=False
working on t=22.400000000000002, exact=False
working on t=25.6, exact=False
working on t=28.799999999999997, exact=False
working on t=32.0, exact=False
working on t=35.2, exact=False
working on t=38.4, exact=False
working on t=41.6, exact=False
working on t=44.800000000000004, exact=False
working on t=48.0, exact=False
working on t=51.2, exact=False
working on t=54.400000000000006, exact=False
working on t=57.599999999999994, exact=False
working on t=60.8, exact=False
working on t=64.0, exact=False
working on t=67.2, exact=False
working on t=70.4, exact=False
working on t=73.60000000000001, exact=False
working on t=76.8, exact=False
working on t=80.0, exact=False
working

In [142]:
import matplotlib as mpl
mpl.rcParams['axes.linewidth'] = 2.0 #set the value globally

font = {'family': 'serif',
        'color':  'k',
        'weight': 'normal',
        'size': 10,
        }

L = ham.Lx
###### plot  spin wave
for i in range(0,deepqd.nspin,1) :
    plt.scatter(np.array(time) * args.Jx, spin_density[:,i], marker = '.')#, label='spin '+str(i)+'deepqd')
    try:
        plt.plot(np.array(time_cls) * args.Jx, spin_density_cls[:,i],label=str(i))#, label='spin '+str(i)+'vanilla')
    except:
        pass
    try:
        plt.plot(ref_Jt, ref_sdw_avg.transpose()[i])
    except:
        pass
plt.xlabel(r'$Jt$',fontdict=font)
plt.ylabel(r'$\langle\sigma_i(t)\rangle$',fontdict=font)
plot_dir  = "./output/{}/l{}T{}".format( args.experiment_name, depth,T) + out_appdix
plt.savefig(plot_dir,dpi=150)
plt.close()

### sdw at the boundary
idx_list = [0, L//2-1, L//2, -1, L//4-1, L//4, L//4-1+L//2, L//4+L//2]
color_list = ['b','c','k','g','m','r','y','purple']
marker = ['x','.','x','.','x','.','x','.']
for idx, i in  enumerate(idx_list):
    x = args.Jx * np.array(time)
    sparse_filter = (np.arange(x.shape[0]) % 5) == 0
    ref_spin_density = np.roll(ref_sdw_avg,L//4,axis=1)
    plt.plot(ref_Jt, ref_spin_density[:,i], 
             color=color_list[idx],linewidth=3)
    plt.scatter( x[sparse_filter], spin_density[:,i][sparse_filter], 
                s = np.ones_like(sparse_filter)*169,
                marker = marker[idx],
                color=color_list[idx], label=r'$s_{'+str((i-8)%32+1)+'}$')
    

plt.xlabel(r'$Jt$',fontdict=font)
plt.ylabel(r'$\langle\sigma_i(t)\rangle$',fontdict=font)
ax = plt.gca()
ax.tick_params(direction='in',labelsize=28)
lgd = plt.legend(fontsize=16,bbox_to_anchor=(1, 1), loc='upper left', ncol=1)

plot_dir  = "./output/{}/boundary_sdw_l{}T{}".format( args.experiment_name, depth,T) + out_appdix
plt.savefig(plot_dir,dpi=150,bbox_extra_artists=(lgd,), bbox_inches='tight')
plt.close()



##### predicted sdw
plt.subplot(311)
ax = plt.gca()
plt.imshow(np.roll(spin_density.transpose(),-8,axis=0), cmap='viridis', 
           interpolation='nearest',
           aspect = 0.25,
           #aspect =spin_density.shape[0]/spin_density.shape[1],
          extent=[0,10,0,L+1])
# plt.xlabel(r'$Jt$')
#     plt.ylabel(r'$\langle\sigma_i(t)\sigma_{i+d}(t)\rangle$')
plt.ylabel(r'$i$',fontdict=font)
plt.xticks([], [])
plt.yticks([0,10,20,30], [0,10,20,30])
ax.tick_params(direction='in',labelsize=10)
cbar = plt.colorbar()
cbar.ax.tick_params(axis='y',direction='in',labelsize=8)

##### reference sdw
plt.subplot(312)
ax = plt.gca()
plt.imshow(ref_sdw_avg.transpose(), cmap='viridis', interpolation='nearest',
           #aspect =ref_sdw_avg.shape[0]/ref_sdw_avg.shape[1],
          aspect = 0.25,
          extent=[0,10,0,L+1])
# plt.xlabel(r'$Jt$')
plt.ylabel(r'$i$',fontdict=font)
plt.xticks([], [],fontname="Times")
plt.yticks([0,10,20,30], [0,10,20,30])
ax.tick_params(direction='in',labelsize=10)
cbar = plt.colorbar()
cbar.ax.tick_params(axis='y',direction='in',labelsize=8)

###### mirror correlation
plt.subplot(313)
ax = plt.gca()
szsz = spin_density[:,:L//2] * np.flip(spin_density[:,L//2:],1)
mirror_corr = mirror_szsz-szsz
mirror_corr = (mirror_corr[:,:L//4] + np.flip(mirror_corr[:,L//4:],1))/2
plt.imshow(mirror_corr.transpose(), cmap='gray', interpolation='nearest',
           aspect =1.025,extent=[0,10,L//4+0.5,0.5])
plt.xlabel(r'$Jt$',fontdict=font)
plt.ylabel(r'$i$',fontdict=font)

plt.xticks(np.arange(0,11,2), np.arange(0,11,2))
plt.yticks(np.arange(1,9,2), np.arange(8,0,-2))

ax.tick_params(direction='in',labelsize=10)
cbar = plt.colorbar()
cbar.ax.tick_params(axis='y',direction='in',labelsize=8)
    
plot_dir  = "./output/{}/SDW_l{}T{}".format( args.experiment_name, depth,T) + out_appdix
plt.savefig(plot_dir,dpi=150)
plt.close()



# ###### averaged correlation
# szsz = np.zeros_like(nn_coupling)
# for d in range(ham.Lx//2):
#     szsz[:,d] = (spin_density * np.roll(spin_density,d+1,axis=1)).mean(1)
# plt.imshow((nn_coupling-szsz).transpose(), cmap='hot', interpolation='nearest',aspect ='auto')
# plt.xlabel(r'$Jt$')
# #     plt.ylabel(r'$\langle\sigma_i(t)\sigma_{i+d}(t)\rangle$')
# plt.ylabel(r'$d$')
# plt.colorbar()
# plt.legend()
# plot_dir  = "./output/{}/correlation_l{}T{}".format( args.experiment_name, depth,T) + out_appdix
# plt.savefig(plot_dir,dpi=150)
# plt.close()

# ###### averaged szsz
# plt.imshow(nn_coupling.transpose(), cmap='hot', interpolation='nearest',aspect ='auto')
# plt.xlabel(r'$Jt$')
# #     plt.ylabel(r'$\langle\sigma_i(t)\sigma_{i+d}(t)\rangle$')
# plt.ylabel(r'$d$')
# plt.colorbar()
# plt.legend()
# plot_dir  = "./output/{}/szsz_l{}T{}".format( args.experiment_name, depth,T) + out_appdix
# plt.savefig(plot_dir,dpi=150)
# plt.close()

In [140]:
### sdw at the boundary
idx_list = [0, L//2-1, L//2, -1, L//4-1, L//4, L//4-1+L//2, L//4+L//2]
color_list = ['b','c','k','g','m','r','y','purple']
marker = ['x','.','x','.','x','.','x','.']
for idx, i in  enumerate(idx_list):
    x = args.Jx * np.array(time)
    sparse_filter = (np.arange(x.shape[0]) % 5) == 0
    ref_spin_density = np.roll(ref_sdw_avg,L//4,axis=1)
    plt.plot(ref_Jt, ref_spin_density[:,i], 
             color=color_list[idx],linewidth=2)
    plt.scatter( x[sparse_filter], spin_density[:,i][sparse_filter], 
#                 s = np.ones_like(sparse_filter)*169,
                marker = marker[idx],
                color=color_list[idx], label=r'$s_{'+str((i-8)%32+1)+'}$')
    
plt.xlabel(r'$Jt$',fontdict=font)
plt.ylabel(r'$\langle\sigma_i(t)\rangle$',fontdict=font)
fig = plt.gcf()
fig.set_size_inches(4,3)
ax = plt.gca()
ax.tick_params(direction='in',labelsize=14)
lgd = plt.legend(fontsize=16,bbox_to_anchor=(1, 1), loc='upper left', ncol=1)
plot_dir  = "./output/{}/boundary_sdw_l{}T{}".format( args.experiment_name, depth,T) + out_appdix
plt.savefig(plot_dir,dpi=150,bbox_extra_artists=(lgd,), bbox_inches='tight')
plt.close()

####CGSP spectrum
x = deepqd.root.net.target_bands.cpu().numpy()
y = deepqd.root.net.norm.cpu().numpy()
plt.bar(x[x<0],y[x<0]**2,width=0.02)
plt.xticks([-0.24,-0.16,-0.08,0], [-0.24,-0.16,-0.08,0])
plt.yticks(np.round(np.arange(5)*0.1,1),np.round(np.arange(5)*0.1,1))
ax = plt.gca()
ax.tick_params(direction='in',labelsize=14)

fig = plt.gcf()
fig.set_size_inches(3,2.5)
plot_dir  = "./output/{}/spectrum_l{}T{}".format( args.experiment_name, depth,T) + out_appdix
plt.savefig(plot_dir,dpi=150)
plt.close()
print(x)
print('Emax={},Emin={},dE={}'.format(x.max(),x.min(),x[2]-x[1]))


[-0.2390625  -0.21718751 -0.1953125  -0.1734375  -0.1515625  -0.1296875
 -0.10781251 -0.0859375  -0.06406251 -0.04218751 -0.0203125   0.00156249
  0.02343749  0.04531248  0.0671875   0.0890625   0.11093749  0.13281249
  0.15468748  0.1765625   0.1984375   0.22031249  0.24218749  0.26406252
  0.2859375   0.3078125   0.32968748  0.3515625   0.37343752  0.3953125
  0.4171875   0.43906248]
Emax=0.4390624761581421,Emin=-0.23906250298023224,dE=0.021875008940696716


In [146]:
net = deepqd.root.net
model_parameters = filter(lambda p: p.requires_grad, net.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print('# params=',params)
dimh =  deepqd.hilbert.dim_hilbert
print(params/dimh)

# params= 7163328
0.011917420896063503


In [36]:
from IPython.display import clear_output
## observe middle layer output of amp-net


deepqd = spectral_tree(args)
deepqd.load_model('0',directory=directory)

full_basis = deepqd.hilbert.get_full_basis()
model = deepqd.root.net
with th.no_grad():
    psi_r, psi_i = model.faithful_forward(full_basis, deepqd.init_state) ## not normalized
psi2 = (psi_r**2+psi_i**2)
psi2normed = psi2 / psi2.sum(-1).unsqueeze(-1)
sparsity = (psi2normed**0.5).sum(-1)
print('sparsity',sparsity)


# model = deepqd.root.net
# s = th.tensor([-1,1]*20).unsqueeze(0)
# s_nb,w_nb = deepqd.hilbert.get_neighbor_list_parallel(s)
# psi_r, psi_i = model.forward(s_nb[0])


# print(th.log(th.abs(psi_r))) 

# s,w=generate_ref_config(100, deepqd.init_state, gamma=0.1)
# for i,q in enumerate(s):
#     print(q)
#     print(w[i])    


##Loaded model from /home/bingjiay/gpfs/ControlQD/nn_data/s16n16-20b16-20MC4000hd128-cell4-lr0.001_skip*4/0.net
# sparsity tensor([50.2663, 53.9543, 54.5530, 40.3695, 64.0339, 47.7522, 69.1760, 79.2141,
#         68.7451, 75.2076, 69.7398, 63.2146, 62.1984, 57.4785, 50.0404, 35.8350],
#        device='cuda:0', grad_fn=<SumBackward1>)

Loaded model from /home/bingjiay/gpfs/ControlQD/nn_data/s16n64-20b64-20MC4000hd128-cell4-lr0.001_skip*4/0.net
sparsity tensor([59.1605, 60.1418, 60.9279, 61.4632, 61.5943, 61.0088, 59.0895, 54.7752,
        47.6569, 44.6676, 50.0274, 57.2877, 62.5243, 64.9267, 64.5550, 59.9240,
        60.3192, 72.8602, 76.4320, 74.5409, 67.0031, 70.4340, 75.8552, 76.8999,
        72.0603, 70.8111, 80.4266, 72.9388, 85.3402, 80.6102, 85.0265, 72.5168,
        83.3271, 75.2956, 66.7001, 74.5933, 80.9364, 79.1272, 79.1645, 75.8233,
        67.1484, 75.4128, 69.7896, 68.6542, 64.5408, 73.3618, 64.6093, 58.2127,
        70.1832, 68.5694, 69.5335, 54.4684, 54.4603, 52.0607, 63.4932, 57.9047,
        50.8800, 52.3663, 52.1555, 48.3553, 38.3298, 38.1471, 38.1514, 49.1094],
       device='cuda:0', grad_fn=<SumBackward1>)


In [49]:
from observer import *
deepqd = spectral_tree(args)
deepqd.load_model('0',directory=directory)
model = deepqd.root.net
s, w = generate_fid_config(4000, model, deepqd.init_state,gamma=0.5)
unique_s = th.unique(s,dim=0)
unique_w = th.unique(w,dim=0)
print(th.unique(s,dim=0).shape)
print(th.unique(w,dim=0).shape)


hist, bin = np.histogram(w.cpu().numpy(), bins=20)
print(bin)
print(hist)


ham = deepqd.hilbert
ref_state = deepqd.init_state.reference_state
nn_mask = th.abs(unique_s - ref_state).sum(-1) == 4
nn_total_mask = th.abs(s - ref_state).sum(-1) == 4
# print(nn_total_mask.sum())
# print(unique_s[nn_mask])
# print(unique_w[nn_mask])
# print(unique_w.max())

source_basis, weight_nbhood = ham.get_neighbor_list_parallel(ref_state.unsqueeze(0))
source_basis = source_basis.reshape(-1,ham.nspin)
sita_evaluated = HamNet(ham, model, source_basis, evaluate_source=True) ## all shapes=(ngroup, batchsize)
ref_evaluated = HamNet(ham, deepqd.init_state, source_basis, evaluate_source=True) ## all shapes=(1, batchsize)
Hpsi_r, Hpsi_i, psi_r, psi_i = faithful_transform(
    sita_evaluated, ref_evaluated, model) ## all shapes=(ngroup, batchsize)

# print('basis',source_basis-ref_state)
# print('sita_r',sita_evaluated[2])
# print('psi_r',psi_r)
# print('Hpsi_r',Hpsi_r)
amp=model._postprocess(model._preprocess(s,for_phase=False),model.amp_net(model._preprocess(s,for_phase=False)))

bandwidth = (model.target_bands[-1]-model.target_bands[0])/(model.groups-1)
scale_factor = (bandwidth/2)**2
# weighted_variance, expH, varH, psipsi=sample_weighted_variance_unified_importance(4000, ham, deepqd.init_state, model)
# print('weighted_variance',weighted_variance/scale_factor)
# print('varH \n ',np.round(th2np(varH/scale_factor),2) )
# print('psipsi',psipsi)

# weighted_variance, expH, exactvarH, psipsi=sample_weighted_variance_unified_importance_exact(ham, deepqd.init_state, model)
# print('exact weighted_variance',weighted_variance/scale_factor)
# print('exact varH \n {}'.format(np.round(th2np((exactvarH-varH)/scale_factor),2)) )
# print('psipsi',psipsi)
# print('l1importance',model.subnet_l1importance())
# psi_r, psi_i = model(ham.get_full_basis())
# th.abs(psi_r).sum(-1)
source_basis = ham.get_full_basis()
# source_basis, weight_nbhood = ham.get_neighbor_list_parallel(ref_state.unsqueeze(0))
# source_basis = source_basis.squeeze(0)
sita_evaluated = HamNet(ham, model, source_basis, evaluate_source=True) ## all shapes=(ngroup, batchsize)
ref_evaluated = HamNet(ham, deepqd.init_state, source_basis, evaluate_source=True) ## all shapes=(1, batchsize)
Hpsi_r, Hpsi_i, psi_r, psi_i = faithful_transform(
    sita_evaluated, ref_evaluated, model) ## all shapes=(ngroup, batchsize)
# print(th.abs(psi_r).sum(0))
norm = (psi_r**2).sum(-1)**0.5
print('norm',norm)

torch.Size([3145, 16])
torch.Size([3145])
[6.7716001e-06 3.2554945e-04 6.4432726e-04 9.6310512e-04 1.2818830e-03
 1.6006608e-03 1.9194387e-03 2.2382166e-03 2.5569943e-03 2.8757721e-03
 3.1945501e-03 3.5133278e-03 3.8321058e-03 4.1508838e-03 4.4696615e-03
 4.7884393e-03 5.1072170e-03 5.4259948e-03 5.7447730e-03 6.0635507e-03
 6.3823285e-03]
[3633  267   26   15   27    4    0    0    0    0    0    0    0    0
    0    0    0    0    0   28]
norm tensor([0.0020, 0.0031, 0.0048, 0.0071, 0.0080, 0.0063, 0.0044, 0.0039, 0.0052,
        0.0104, 0.0245, 0.0252, 0.0129, 0.0201, 0.0345, 0.0131, 0.0113, 0.0840,
        0.0306, 0.0676, 0.1358, 0.0141, 0.1069, 0.1338, 0.0111, 0.1186, 0.1376,
        0.1265, 0.1671, 0.0327, 0.1631, 0.1307, 0.1354, 0.2645, 0.0119, 0.1163,
        0.2462, 0.0970, 0.0222, 0.1997, 0.2510, 0.0158, 0.0872, 0.2288, 0.0439,
        0.2132, 0.0122, 0.0045, 0.2705, 0.1802], device='cuda:0',
       grad_fn=<PowBackward0>)


In [3]:
from util import bit2integer
deepqd = spectral_tree(args)
deepqd.load_tree(directory)
deepqd.load_all_model(directory=directory)

net0 = deepqd.root.net
net0.data_parallel()
net01 = deepqd.root.children[0].net
net01.data_parallel()

net = net01


hilbert = deepqd.hilbert
full_basis= hilbert.get_full_basis()
psi_r, psi_i = net(full_basis)

active_amp_i = deepqd.reassign(th.zeros(deepqd.root.net.groups))
active_amp_r = active_amp_i.clone()

total_psi_r = (psi_r * net.groups_amplitude[:,None]).sum(0)
total_psi_i = (psi_i * net.groups_amplitude[:,None]).sum(0)
total_psi2 = total_psi_r**2 + total_psi_i**2
# net_ref = deepqd.init_state

net_ref = ensemble2ref_wrapper(net0, 
                               expH = net0.expH[1],
                               amp_r = active_amp_r,
                               amp_i = active_amp_i)


0
├── 0-1
└── 0-2


In [43]:
with th.no_grad():
    exp_dict = deepqd.exact_all_expectation(net, net_ref)
    bw = (net.target_bands[1]-net.target_bands[0])
print(
    'varH',exp_dict['variance'] / (bw/2)**2,
    'expH', exp_dict['expH']
)

expHl=[]
varHl=[]
varHl_simple = []
for i in range(100):
    with th.no_grad():
        batchsize = 500
        tol=0.02
        target_bands = net.target_bands[:,None]
        dim_group, dim_batch = 0, 1
        target_bands = net.target_bands[:,None]
        basis_input_var, mcWeight_var = net.generate(batchsize,tol=tol)
        sita_r, sita_i = net(basis_input_var)
        # sita_r, sita_i = self.cheapnet(net, basis_input_var)
        Hsita_r, Hsita_i = deepqd.Hnet(net, basis_input_var)
        ## compute quantum expectaions
        HmEsita_norm2 = (Hsita_r - sita_r * target_bands)**2 + (Hsita_i - sita_i * target_bands)**2
        sita_norm2 = sita_r**2 + sita_i**2
        varH = (HmEsita_norm2 / mcWeight_var).mean(dim_batch) 
        varH_reg = (HmEsita_norm2 / mcWeight_var).mean(dim_batch) / (sita_norm2/mcWeight_var).mean(dim_batch)
        ratio = (
            varH - exp_dict['variance']
                )/(
            varH_reg - exp_dict['variance']
        )
        print("incorrect/correct {}".format(ratio))
        expH = ( (sita_r * Hsita_r + sita_i * Hsita_i) 
                    / mcWeight_var ).mean(dim_batch) / (sita_norm2/mcWeight_var).mean(dim_batch)
        expHl.append(expH)
        varHl.append(varH_reg)
        varHl_simple.append(varH)
print('var_simple', sum(varHl_simple)/100 / (bw/2)**2,'\n',
    'varH',sum(varHl)/100 / (bw/2)**2,'\n',
     'expH', sum(expHl)/100)

varH tensor([2.2620, 1.7883, 2.0908], device='cuda:0', dtype=torch.float64) expH tensor([0.0114, 0.0989, 0.1848], device='cuda:0', dtype=torch.float64)
incorrect/correct tensor([1.0107, 1.0697, 0.9771], device='cuda:0', dtype=torch.float64)
incorrect/correct tensor([1.0591, 0.9959, 0.9820], device='cuda:0', dtype=torch.float64)
incorrect/correct tensor([1.0103, 0.9557, 1.1091], device='cuda:0', dtype=torch.float64)
incorrect/correct tensor([1.0495, 1.0059, 1.1001], device='cuda:0', dtype=torch.float64)
incorrect/correct tensor([1.0847, 0.9695, 1.0242], device='cuda:0', dtype=torch.float64)
incorrect/correct tensor([1.0070, 1.0508, 0.0707], device='cuda:0', dtype=torch.float64)
incorrect/correct tensor([0.8776, 1.0806, 1.0326], device='cuda:0', dtype=torch.float64)
incorrect/correct tensor([1.1670, 1.4129, 1.0098], device='cuda:0', dtype=torch.float64)
incorrect/correct tensor([1.1743, 0.9378, 0.1941], device='cuda:0', dtype=torch.float64)
incorrect/correct tensor([1.0148, 1.0224, 0.719